# Anatomy of ring and motor-related neurons in the hindbrain

In [ ]:
%matplotlib widget
from pathlib import Path
from shutil import copy

import flammkuchen as fl
import lotr.plotting as pltltr
import numpy as np
import pandas as pd
from lotr import A_FISH, LotrExperiment, dataset_folders

COLS = pltltr.COLS
from matplotlib import pyplot as plt
from tqdm import tqdm

## Localization of ring neurons
First, let's have a look at the positions of ring neurons. We will also load information about the motor regressors, as later we will also plot the localizations of the motor-selective neurons.

In [ ]:
Z_SCATTER_FACTOR = 10  # add some random dispersion in z to make all rois visible
data_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    cent_coords = exp.morphed_coords_um
    data_dict = dict()
    for i, lab in enumerate(["s_i", "l_r", "a_p"]):
        data_dict.update({lab: cent_coords[:, i]})
    data_dict["s_i"] = (
        data_dict["s_i"] + np.random.rand(len(data_dict["a_p"])) * Z_SCATTER_FACTOR
    )
    data_dict["fid"] = path.name
    data_dict["hdn"] = ~exp.nonhdn_indexes

    data_df.append(pd.concat([pd.DataFrame(data_dict), exp.motor_regressors], axis=1))

data_df = pd.concat(data_df, ignore_index=1)

First, a nice horizontal view:

In [ ]:
f, ax = plt.subplots(figsize=(3, 3.5))
s = 8
ax.scatter(
    data_df["l_r"], data_df["a_p"], lw=0, s=s, alpha=0.03, color=(0.3,) * 3,
)
loc = [-110, -138]
ax.axis("equal")
pltltr.add_anatomy_scalebar(ax, pos=loc, length=40, fontsize=6)

pltltr.savefig("horview_allneurons_noring")
ax.scatter(
    data_df["l_r"][data_df["hdn"]],
    data_df["a_p"][data_df["hdn"]],
    color=COLS["ring"],
    lw=0,
    s=s,
    alpha=0.1,
)
pltltr.savefig("horview_allneurons")

Then, all projections:

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(7, 2.5))
sel = data_df["hdn"]
planes = "horizontal", "sagittal", "frontal"
locs = [[-110, -138], [-52, -156], [-105, -71]]
for i, coords in enumerate(
    [
        [data_df["l_r"], data_df["a_p"]],
        [data_df["s_i"], data_df["a_p"]],
        [data_df["l_r"], data_df["s_i"]],
    ]
):
    axs[i].scatter(
        coords[0], coords[1], lw=0, s=s, alpha=0.03, color=(0.3,) * 3,
    )
    axs[i].scatter(
        coords[0][sel], coords[1][sel], color=COLS["ring"], lw=0, s=s, alpha=0.1,
    )

    axs[i].axis("equal")
    axs[i].axis("off")
    axs[i].set_title(planes[i].capitalize() + " view")

    pltltr.add_anatomy_scalebar(
        axs[i], plane=planes[i], pos=locs[i], length=30, fontsize=6
    )

pltltr.savefig("allviews_allneurons")

## Motor selectivity

Now we look at how much motor selective are neurons

In [ ]:
f, ax = plt.subplots(figsize=(3.5, 4))
ax.scatter(
    data_df["l_r"], data_df["a_p"], lw=0, s=s, alpha=0.05, color=(0.3,) * 3,
)
thr = 0.7
s = 10
for reg, regnot, c in zip(
    ["right_1", "left_1"],
    ["left_1", "right_1"],
    [COLS["sides"]["rt"], COLS["sides"]["lf"]],
):
    motor_sel = (data_df[reg] > thr) & (data_df[regnot] < thr)
    lab = reg.split("_")[0]
    ax.scatter(
        data_df["l_r"][motor_sel],
        data_df["a_p"][motor_sel],
        # c=data_df[reg][motor_sel],
        label=f"mot. {lab} > {thr}",
        color=c,
        # cmap=cmap,
        lw=0,
        vmin=0.0,
        vmax=1,
        s=s,
        alpha=0.3,
    )
loc = [-110, -138]
ax.axis("equal")
ax.set_title(planes[i].capitalize() + " view")
l = ax.legend(loc=2, bbox_to_anchor=(0.8, -0.2))
for lh in l.legendHandles:
    lh.set_alpha(1)
pltltr.add_anatomy_scalebar(ax, pos=loc, length=30, fontsize=6)
pltltr.savefig("horview_motorneurons")

ax.scatter(
    data_df["l_r"][data_df["hdn"]],
    data_df["a_p"][data_df["hdn"]],
    color=COLS["ring"],
    lw=0,
    s=s,
    alpha=0.1,
    label="ring",
)
l = ax.legend(loc=2, bbox_to_anchor=(0.7, 0.2))
for lh in l.legendHandles:
    lh.set_alpha(1)
pltltr.savefig("horview_motorneurons_andring")

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(7, 2.5))
s = 8
sel = data_df["hdn"]
planes = "horizontal", "sagittal", "frontal"
locs = [[-110, -138], [-52, -156], [-105, -71]]
for i, coords in enumerate(
    [
        [data_df["l_r"], data_df["a_p"]],
        [data_df["s_i"], data_df["a_p"]],
        [data_df["l_r"], data_df["s_i"]],
    ]
):
    axs[i].scatter(
        coords[0], coords[1], lw=0, s=s, alpha=0.03, color=(0.3,) * 3,
    )
    for reg, regnot, c in zip(
        ["right_1", "left_1"],
        ["left_1", "right_1"],
        [COLS["sides"]["rt"], COLS["sides"]["lf"]],
    ):
        motor_sel = (data_df[reg] > thr) & (data_df[regnot] < thr)
        lab = reg.split("_")[0]
        axs[i].scatter(
            coords[0][motor_sel],
            coords[1][motor_sel],
            # c=data_df[reg][motor_sel],
            label=f"mot. {lab} > {thr}",
            color=c,
            # cmap=cmap,
            lw=0,
            vmin=0.0,
            vmax=1,
            s=s,
            alpha=0.3,
        )

    axs[i].axis("equal")
    axs[i].axis("off")
    axs[i].set_title(planes[i].capitalize() + " view")

    pltltr.add_anatomy_scalebar(
        axs[i], plane=planes[i], pos=locs[i], length=30, fontsize=6
    )

pltltr.savefig("allviews_allneurons_motor")

Contour masks for a fish

In [ ]:
exp = LotrExperiment(A_FISH)

In [ ]:
rois = exp.rois_stack

ring_rois = exp.color_rois_by(np.ones(exp.n_hdns))

In [ ]:
proj = ring_rois.max(0)[:, :, 0]

In [ ]:
proj.shape

In [ ]:
plt.figure(facecolor="none")
plt.contour(proj, levels=1, colors=[COLS["ring"]], linewidths=1)
plt.axis("equal")
plt.axis("off")
pltltr.savefig("rois_mask")